In [1]:
import warnings
warnings.simplefilter(action='ignore')

import pandas as pd
import pandasql
import psycopg2
from pandasql import sqldf
from sqlalchemy import create_engine

In [5]:
# pandas dataframes of CSV files for reference

diagnosis_df = pd.read_csv('iris_sample_diagnosis_202003061131.csv')
history_df = pd.read_csv('iris_sample_diagnosis_history_202003061132.csv')
insurance_df = pd.read_csv('iris_sample_diagnosis_insurance_202003061132.csv')
procedures_df = pd.read_csv('iris_sample_diagnosis_procedures_202003061133.csv')
patient_df = pd.read_csv('iris_sample_patient_202003061133.csv')

# 0.

Import all tables into SQL. There are 5 tables: Insurance, Procedures, Patient (demographics), Diagnosis and Diagnosis (dates of the Diagnoses) , variables should be self-explanatory, if it ends in ID it is either a primary key ID or some unique identifier to identity a patient or procedure or diagnosis or diagnosis history record.

### Generating the database

I generated a local PostgreSQL database and copied in the CSV files by using the **psycopg2** package. I instantiated the connection and created a cursor object to make commands against the database.

### Adjustments and assumptions


The original diagnosis CSV file contained extra commas in lines 1477, 1603, 6100, 7704, 8094, 9118, 9806, 11568, 13413, and 13744, which were incorrectly recognized as delimiters when attempting my method to copy from CSV files to SQL tables. This always occured in the last column. For now, I resolved this by manually replacing the extra commas with semi-colons. In addition, line 13196 contained an \ element which caused UTF errors, to resolve this I simply removed the \ element.

The original procedures CSV file had similar issues as the diagnosis CSV file in lines 82, 2486, 4455, 4862, 6125, 6556, 6622, 8817, 9052, 9102, 11407, 12271, 16861, 18107, 18255, 20133, 20261, 20755, 20946, 21765. I also resolved this by replacing commas with semi-colons

The original insurances CSV file had similar issues as the diagnosis CSV file in lines 73, 85, 168, 218, 224, 300, 314, 325, 326, 372, 469, 498, 547, 670, 679, 686, 709, 767, 822. I also resolved this by replacing commas with semi-colons

When setting conditional expressions on codes, I assumed the codes needed to be **precisely** identical to the listed ICD-9/10 codes and CPT. 

In [2]:
# Instantiate connection to local database; Parameters were eventually removed for security reasons
conn = psycopg2.connect("host=* dbname=* user=* password=*")

# Create a cursor object to make commands against the local database
cur = conn.cursor()

# Start from scratch and create all the five tables
cur.execute("""
  DROP TABLE IF EXISTS patients;
  CREATE TABLE patients(
  patient_guid text PRIMARY KEY
, gender text
, year_of_birth integer
, week_of_birth integer
, ethnicity text
, race text
);

  DROP TABLE IF EXISTS diagnoses;
  CREATE TABLE diagnoses(
  patient_diagnosis_id text
, patient_guid text
, dx_code text
);

  DROP TABLE IF EXISTS procedures;
  CREATE TABLE procedures(
  patient_procedure_id text
, patient_guid text
, patient_visit_id text
, proc_date date
, proc_code text
);

  DROP TABLE IF EXISTS insurances;
  CREATE TABLE insurances(
  patient_guid text
, insurance_id integer
, ins_type text
);

  DROP TABLE IF EXISTS histories;
  CREATE TABLE histories(
  patient_diagnosis_history_id text
, patient_diagnosis_id text
, patient_visit_id text
, patient_guid text
, dx_date text
);
""")

# Make a function to copy a CSV file to a table from our database
def copy_csv_to_table(csv, table):
    with open(csv, 'r') as f:
        next(f) # Skip the header row.
        cur.copy_from(f, table, sep=',', null='')

# Make list of relevant csv files and table names as iterables
csv_files = ['iris_sample_patient_202003061133.csv',            #patients
            'iris_sample_diagnosis_202003061131.csv',           #diagnoses
            'iris_sample_diagnosis_procedures_202003061133.csv',#procedures
            'iris_sample_diagnosis_insurance_202003061132.csv', #insurances
            'iris_sample_diagnosis_history_202003061132.csv']   #histories

table_names = ['patients', 
               'diagnoses', 
               'procedures', 
               'insurances', 
               'histories']

# Copy CSV files into the five tables
for csv, table in zip(csv_files, table_names):
    copy_csv_to_table(csv, table)
conn.commit()

In [16]:
# Close connection when done 
conn.close()

In [3]:
# Give the database a test run and compare with pandas dataframe...
pd.read_sql("""SELECT *
               FROM patients
               LIMIT 5""", conn)

,patient_guid,gender,year_of_birth,week_of_birth,ethnicity,race
0,"""01726e62bcef4e66a94e026eaba37b3a""",Male,1957,40,Not Hispanic or Latino,Caucasian
1,"""07a2141aa57a4985b320038e53716b70""",Male,1951,35,Not Hispanic or Latino,Caucasian
2,"""12546c7b10b74cc88679bed298ebce26""",Male,1941,48,Hispanic or Latino,Caucasian
3,"""03657a9025e94c3bb6e4a3c5a4a3cac5""",Male,1958,51,Not Hispanic or Latino,Declined to answer
4,"""0a293701c318490ab76b21b33d1a8264""",Male,1964,44,Not Hispanic or Latino,Caucasian


In [7]:
patient_df.head(5)

,patient_guid,gender,year_of_birth,week_of_birth,ethnicity,race
0,01726e62bcef4e66a94e026eaba37b3a,Male,1957.0,40.0,Not Hispanic or Latino,Caucasian
1,07a2141aa57a4985b320038e53716b70,Male,1951.0,35.0,Not Hispanic or Latino,Caucasian
2,12546c7b10b74cc88679bed298ebce26,Male,1941.0,48.0,Hispanic or Latino,Caucasian
3,03657a9025e94c3bb6e4a3c5a4a3cac5,Male,1958.0,51.0,Not Hispanic or Latino,Declined to answer
4,0a293701c318490ab76b21b33d1a8264,Male,1964.0,44.0,Not Hispanic or Latino,Caucasian


The CSV data was imported succesfully to the local database. The only issue was how double quotes were kept in the local database (See *patient_guid* ), whereas they do not show up when pulled with pandas.

In order to remove any chance of error since the queries call for joining tables, I make good use of the TRIM function, that is available in PostgreSQL, on relevant *text* columns to be be sure quotes are never considered. I applied this to all *text* columns on all tables.

In [8]:
# No double quotes!
pd.read_sql("""SELECT
                 TRIM('"' FROM patient_guid) AS patient_guid,
                 TRIM('"' FROM gender) AS gender,
                 year_of_birth,
                 week_of_birth,
                 TRIM('"' FROM ethnicity) AS ethnicity,
                 TRIM('"' FROM race) AS race
                 
               FROM patients
               LIMIT 5""", conn)

,patient_guid,gender,year_of_birth,week_of_birth,ethnicity,race
0,01726e62bcef4e66a94e026eaba37b3a,Male,1957,40,Not Hispanic or Latino,Caucasian
1,07a2141aa57a4985b320038e53716b70,Male,1951,35,Not Hispanic or Latino,Caucasian
2,12546c7b10b74cc88679bed298ebce26,Male,1941,48,Hispanic or Latino,Caucasian
3,03657a9025e94c3bb6e4a3c5a4a3cac5,Male,1958,51,Not Hispanic or Latino,Declined to answer
4,0a293701c318490ab76b21b33d1a8264,Male,1964,44,Not Hispanic or Latino,Caucasian


# 1. 

Determine the number of patients who were diagnosed with RRD (Rhegmatogenous retinal detachment) between Jan 1, 2013 and Dec 31, 2018. 
RRD diagnoses can be identified by these ICD DX codes:

ICD-9 = 361.00, 361.01, 361.02, 361.03, 361.04, 361.05, 361.06, 361.07

ICD-10 = H33.001, H33.002, H33.003, H33.009, H33.011, H33.012, H33.013, H33.019, H33.021, H33.022, H33.023, H33.029, H33.031, H33.032, H33.033, H33.039, H33.041, H33.042, H33.043, H33.049, H33.051, H33.052, H33.053, H33.059, H33.8.

Use regular expressions or 'ilike' and respective syntax coding in the where clause when querying for the ICD DX codes and use dx date for the date restriction.

In [9]:
# Can we use ILIKE '361.%' for ICD-9 codes?
SQL1_361 = """
SELECT 
  DISTINCT TRIM('"' FROM dx_code) AS dx_code
FROM diagnoses
WHERE TRIM('"' FROM dx_code) ILIKE '361.%'
ORDER BY 1
"""
pd.read_sql(SQL1_361, conn)
# It looks like we cannot, since there are codes that start with '361.' but are unrelated to RRD. 
# e.g. 361.33

,dx_code
0,361.00
1,361.01
2,361.02
3,361.03
4,361.04
5,361.05
6,361.06
7,361.07
8,361.10
9,361.12


In [10]:
# Can we use ILIKE 'H33%' for ICD-10 codes?
SQL1_H33 = """
SELECT 
  DISTINCT TRIM('"' FROM dx_code) AS dx_code
FROM diagnoses
WHERE TRIM('"' FROM dx_code) ILIKE 'H33%'
ORDER BY 1
"""
pd.read_sql(SQL1_H33, conn)
# It looks like we cannot in this case either, since there are codes that start with 'H33' but are unrelated to RRD.
# e.g. H33.10

,dx_code
0,H33.00
1,H33.001
2,H33.002
3,H33.003
4,H33.009
5,H33.01
6,H33.011
7,H33.012
8,H33.013
9,H33.019


A single *patient_guid* can have multiple *patient_diagnosis_ids*, all within the given diagnosis and date constraints. To get the unique count of patients, I used the COUNT and DISTINCT functions.

To connect *dx_code* to its respective *dx_date*, I joined the diagnoses and histories tables on the *patient_guid* and *patient_diagnosis_id* columns.

In [11]:
# Specify the dx_code constraints manually using the IN clause 
# Specify the dx_date constraints using the BETWEEN clause
# Use COUNT DISTINCT to get unique number of relevant patients
# Join the diagnoses and histories tables on the patient_guid and patient_diagnosis_id columns
SQL1 = """
SELECT 
  COUNT(DISTINCT(TRIM('"' FROM diagnoses.patient_guid))) as num_patients
FROM diagnoses

LEFT JOIN histories ON TRIM('"' FROM histories.patient_guid) = TRIM('"' FROM diagnoses.patient_guid)
  AND TRIM('"' FROM histories.patient_diagnosis_id) = TRIM('"' FROM diagnoses.patient_diagnosis_id)

WHERE TRIM('"' FROM diagnoses.dx_code) IN 
('361.00', '361.01', '361.02', '361.03', '361.04', '361.05', '361.06', '361.07',
 'H33.001', 'H33.002', 'H33.003', 'H33.009', 'H33.011', 'H33.012', 'H33.013', 'H33.019', 
 'H33.021', 'H33.022', 'H33.023', 'H33.029', 'H33.031', 'H33.032', 'H33.033', 'H33.039', 
 'H33.041', 'H33.042', 'H33.043', 'H33.049', 'H33.051', 'H33.052', 'H33.053', 'H33.059', 
 'H33.8')
AND histories.dx_date BETWEEN '2013-01-01' AND '2018-12-31'
"""
pd.read_sql(SQL1, conn)
# There are 230 patients that meet the conditions

,num_patients
0,230


# 2. 

Determine the number of patients from #1. above who had an RRD surgical procedure between Jan 1, 2013 and December 31, 2018. RRD surgical procedures can be identified by CPT proc codes 67101, 67105, 67107, 67108, 67112, 67113, 67145, 67141.
Use 'ilike' where clause when querying for the CPT codes.

In [12]:
# Can we use ILIKE for the CPT proc codes?
SQL2_671 = """
SELECT 
  DISTINCT TRIM('"' FROM proc_code) 
FROM procedures
WHERE TRIM('"' FROM proc_code) ILIKE '671%'
"""
pd.read_sql(SQL2_671, conn)
# It looks like we cannot in this case either, since there are codes that start with '671' 
# but are unrelated to RRD surgical procedures.
# e.g 67121

,btrim
0,67101
1,67105
2,67107
3,671077
4,67108
5,6710880
6,67108A
7,67108H
8,67108SG
9,67110


Since we are looking within an output of a previous query, it's a good idea to use a CTE (Common Table Expression) to easily apply a previously used query.

A single *patient_guid* can have multiple *patient_procedure_ids*, all within the given procedures and date constraints. To get the unique count of patients, I used the DISTINCT function.

In [13]:
# Use CTE (WITH...AS) to apply a previously used query
# Annotations within SQL queries are prefixed with --
SQL2 = """
---CTEs---
--CTE of all relevant RRD patient_guids
WITH rrd_patients AS (
SELECT 
  DISTINCT(TRIM('"' FROM diagnoses.patient_guid)) as patient_guid
FROM diagnoses

LEFT JOIN histories 
  ON TRIM('"' FROM histories.patient_guid) = TRIM('"' FROM diagnoses.patient_guid)
  AND TRIM('"' FROM histories.patient_diagnosis_id) = TRIM('"' FROM diagnoses.patient_diagnosis_id)

WHERE TRIM('"' FROM diagnoses.dx_code) IN 
('361.00', '361.01', '361.02', '361.03', '361.04', '361.05', '361.06', '361.07',
 'H33.001', 'H33.002', 'H33.003', 'H33.009', 'H33.011', 'H33.012', 'H33.013', 'H33.019', 
 'H33.021', 'H33.022', 'H33.023', 'H33.029', 'H33.031', 'H33.032', 'H33.033', 'H33.039', 
 'H33.041', 'H33.042', 'H33.043', 'H33.049', 'H33.051', 'H33.052', 'H33.053', 'H33.059', 
 'H33.8')
AND histories.dx_date BETWEEN '2013-01-01' AND '2018-12-31'
)
---------

SELECT COUNT(DISTINCT(TRIM('"' FROM rrd_patients.patient_guid))) as num_procedures
FROM rrd_patients
LEFT JOIN procedures 
 ON TRIM('"' FROM rrd_patients.patient_guid) = TRIM('"' FROM procedures.patient_guid)
WHERE TRIM('"' FROM procedures.proc_code) IN ('67101', '67105', '67107', '67108', '67112', '67113', '67145', '67141')
AND procedures.proc_date BETWEEN '2013-01-01' AND '2018-12-31'
"""
pd.read_sql(SQL2, conn)
# There are 209 patients that meet the conditions

,num_procedures
0,209


# 3.

Obtain the counts by age, gender, and race, and insurance (take the mode as patients have multiple) of the individuals identified above (from #2).
Report age from the first date of procedure (proc_date) per patient.  Then, stratify the counts by year of procedure.

Since we are aiming to refer from a previous query, using CTEs is once again ideal.

To obtain the mode of insurance per patient, I used the *mode()* ordered-set aggregate function, which returns the most frequent value per group.

To obtain the age, I used the *TO_DATE()* function, which is able to take in a given year and week number, and output the Monday date of that given year/week (a small assumption), and then took the difference of that date and the first procedure date using the *age()* function and extracting the number of years using *DATE_PART()*.  

### NOTE:
In cases where there are equally frequent results per patient, the *mode()* function outputs the first value that appears from top to bottom.  

In [14]:
# Use mode() WITHIN GROUP to get mode of ins_type per patient
# Use TO_DATE(), age(), and DATE_PART() to obtain age at first procedure
# Annotations within SQL queries are prefixed with --
SQL3_age = """
---CTEs---

--CTE of all relevant RRD patient_guids
WITH rrd_patients AS (
SELECT 
  DISTINCT(TRIM('"' FROM diagnoses.patient_guid)) as patient_guid  
FROM patients

LEFT JOIN diagnoses 
  ON TRIM('"' FROM diagnoses.patient_guid) = TRIM('"' FROM patients.patient_guid)
LEFT JOIN histories 
  ON TRIM('"' FROM histories.patient_guid) = TRIM('"' FROM diagnoses.patient_guid)
  AND TRIM('"' FROM histories.patient_diagnosis_id) = TRIM('"' FROM diagnoses.patient_diagnosis_id)

WHERE TRIM('"' FROM diagnoses.dx_code) IN 
('361.00', '361.01', '361.02', '361.03', '361.04', '361.05', '361.06', '361.07',
 'H33.001', 'H33.002', 'H33.003', 'H33.009', 'H33.011', 'H33.012', 'H33.013', 'H33.019', 
 'H33.021', 'H33.022', 'H33.023', 'H33.029', 'H33.031', 'H33.032', 'H33.033', 'H33.039', 
 'H33.041', 'H33.042', 'H33.043', 'H33.049', 'H33.051', 'H33.052', 'H33.053', 'H33.059', 
 'H33.8')
AND histories.dx_date BETWEEN '2013-01-01' AND '2018-12-31'

--CTE of all RRD patient_guids that also underwent an RRD procedure
), rrd_procedures AS (

SELECT 
  DISTINCT(TRIM('"' FROM rrd_patients.patient_guid)) as patient_guid
FROM rrd_patients
LEFT JOIN procedures 
 ON TRIM('"' FROM rrd_patients.patient_guid) = TRIM('"' FROM procedures.patient_guid)
WHERE TRIM('"' FROM procedures.proc_code) IN ('67101', '67105', '67107', '67108', '67112', '67113', '67145', '67141')
AND procedures.proc_date BETWEEN '2013-01-01' AND '2018-12-31'

--CTE of all RRD patient_guids that also underwent an RRD procedure and their first RRD procedure dates 
), first_proc_dates AS (
SELECT 
  TRIM('"' FROM rrd_patients.patient_guid) AS patient_guid, 
  MIN(proc_date) AS first_proc_date
FROM rrd_patients
LEFT JOIN procedures 
 ON TRIM('"' FROM rrd_patients.patient_guid) = TRIM('"' FROM procedures.patient_guid)
WHERE TRIM('"' FROM procedures.proc_code) IN 
('67101', '67105', '67107', '67108', '67112', '67113', '67145', '67141')
AND procedures.proc_date BETWEEN '2013-01-01' AND '2018-12-31'
GROUP BY 1

--CTE of patients and their insurance that appears most frequently
), insurance_mode AS (
SELECT 
  TRIM('"' FROM insurances.patient_guid) AS patient_guid, 
  mode() WITHIN GROUP (ORDER BY insurances.ins_type) AS ins_type
FROM insurances
GROUP BY 1

--CTE of relevant CTEs joined, with age_at_first_proc included
), patient_demo AS (
SELECT 
  patients.*, 
  insurance_mode.ins_type, 
  first_proc_dates.first_proc_date::date,
  DATE_PART('year',age(first_proc_dates.first_proc_date, 
                   TO_DATE(CONCAT(patients.year_of_birth, patients.week_of_birth), 'IYYYIW'))) AS age_at_first_proc
FROM patients
LEFT JOIN insurance_mode 
  ON TRIM('"' FROM patients.patient_guid) = TRIM('"' FROM insurance_mode.patient_guid)
LEFT JOIN first_proc_dates
  ON TRIM('"' FROM first_proc_dates.patient_guid) = TRIM('"' FROM patients.patient_guid)
 
WHERE TRIM('"' FROM patients.patient_guid) IN (SELECT TRIM('"' FROM first_proc_dates.patient_guid) 
                                               FROM first_proc_dates)

--CTE of previous CTEs joined and primed for aggregating, with age_at_first_proc BUCKETED 
), patient_demo_v2 AS (
SELECT 
  patient_demo.*,
  CASE 
   WHEN age_at_first_proc BETWEEN 0 AND 10 THEN '0-10'
   WHEN age_at_first_proc BETWEEN 11 AND 20 THEN '11-20'
   WHEN age_at_first_proc BETWEEN 21 AND 30 THEN '21-30'
   WHEN age_at_first_proc BETWEEN 31 AND 40 THEN '31-40'
   WHEN age_at_first_proc BETWEEN 41 AND 50 THEN '41-50'
   WHEN age_at_first_proc BETWEEN 51 AND 60 THEN '51-60'
   WHEN age_at_first_proc BETWEEN 61 AND 70 THEN '61-70'
   WHEN age_at_first_proc BETWEEN 71 AND 80 THEN '71-80'
   WHEN age_at_first_proc > 80 THEN '81+'
  END AS age_interval
FROM patient_demo
)

---------
SELECT 
  age_interval,
  gender,
  race,
  ins_type,
  EXTRACT('year' FROM first_proc_date) AS first_proc_year,
  COUNT(*)
  
FROM patient_demo_v2
GROUP BY 1,2,3,4,5
ORDER BY 1,2,3,4,5
"""
pd.read_sql(SQL3_age, conn)

,age_interval,gender,race,ins_type,first_proc_year,count
0,11-20,Female,Asian,None,2014.0,1
1,21-30,Female,Black or African American,Blue Cross/Blue Shield,2015.0,1
2,21-30,Female,Caucasian,Medicaid,2017.0,1
3,21-30,Female,Caucasian,Private Health Insurance,2013.0,1
4,21-30,Male,Black or African American,Military / Department of Defense/ TRICARE,2017.0,1
5,21-30,Male,Caucasian,Blue Cross/Blue Shield,2014.0,1
6,21-30,Male,Caucasian,Medicare Managed Care (Advantage Plans),2017.0,1
7,31-40,Female,Caucasian,Private Health Insurance,2015.0,2
8,31-40,Female,Caucasian,Private Health Insurance,2017.0,1
9,31-40,Female,Caucasian,Private Health Insurance,2018.0,1


# 4. 

Obtain the number of patients in a table from the patients from #1 above who ALSO had an RRD surgical procedure AFTER the date of their diagnosis of RRD.

As concluded earlier, a single *patient_guid* can have multiple *patient_diagnosis_ids* and *patient_procedure_ids* with multiple associated dates (*dx_date* and *proc_date*). To get all patients that had an RRD procedure AFTER their diagnosis date, their earliest *dx_dates* and *proc_dates* need to be compared (I moved forward assuming that their first RRD diagnosis date and first RRD procedure date were to be compared). 

More specifically, we are looking for patients who have have an RRD diagnosis and RRD procedure, where their earliest *proc_date* is later than their earliest *dx_date*. The earliest dates per patient can be captured using GROUP BY MIN functions and then comparing the dates.

In [15]:
# Start with bulk of query used in #1, it was formatted in order to easily extract and compare all dx_dates
# Use more CTEs and GROUP BY MIN to create Tables that show earliest dx_date and proc_date per RRD patient
# Annotations within SQL queries are prefixed with --

SQL4 = """
---CTEs---

--CTE of all relevant RRD patient_guids
WITH rrd_patients AS (
SELECT 
  TRIM('"' FROM diagnoses.patient_guid) as patient_guid,
  TRIM('"' FROM diagnoses.dx_code) as dx_code,
  histories.dx_date
FROM diagnoses

LEFT JOIN histories 
  ON TRIM('"' FROM histories.patient_guid) = TRIM('"' FROM diagnoses.patient_guid)
  AND TRIM('"' FROM histories.patient_diagnosis_id) = TRIM('"' FROM diagnoses.patient_diagnosis_id)

WHERE TRIM('"' FROM diagnoses.dx_code) IN 
('361.00', '361.01', '361.02', '361.03', '361.04', '361.05', '361.06', '361.07',
 'H33.001', 'H33.002', 'H33.003', 'H33.009', 'H33.011', 'H33.012', 'H33.013', 'H33.019', 
 'H33.021', 'H33.022', 'H33.023', 'H33.029', 'H33.031', 'H33.032', 'H33.033', 'H33.039', 
 'H33.041', 'H33.042', 'H33.043', 'H33.049', 'H33.051', 'H33.052', 'H33.053', 'H33.059', 
 'H33.8')
AND histories.dx_date BETWEEN '2013-01-01' AND '2018-12-31'


--CTE of all RRD patient_guids and their first RRD diagnoses dates
), first_dx_dates AS (
SELECT 
  patient_guid, 
  MIN(dx_date) AS first_dx_date
FROM rrd_patients
GROUP BY 1


--CTE of all RRD patient_guids that also underwent an RRD procedure and their first RRD procedure dates
), first_proc_dates AS (
SELECT 
  rrd_patients.patient_guid, 
  MIN(proc_date) AS first_proc_date
FROM rrd_patients
LEFT JOIN procedures ON TRIM('"' FROM rrd_patients.patient_guid) = TRIM('"' FROM procedures.patient_guid)
WHERE TRIM('"' FROM procedures.proc_code) IN 
('67101', '67105', '67107', '67108', '67112', '67113', '67145', '67141')
GROUP BY 1


--CTE of RRD patient_guids with first_dx_dates and first_proc_dates joined
), dates_joined AS (
SELECT 
  first_dx_dates.patient_guid, 
  first_dx_dates.first_dx_date::date, 
  first_proc_dates.first_proc_date::date
FROM first_dx_dates
LEFT JOIN first_proc_dates ON TRIM('"' FROM first_dx_dates.patient_guid) = TRIM('"' FROM first_proc_dates.patient_guid)
WHERE first_dx_dates.first_dx_date::date < first_proc_dates.first_proc_date::date
)
----------
SELECT COUNT(*) FROM dates_joined
"""
pd.read_sql(SQL4, conn)
# There are 105 patients that meet the conditions

,count
0,105
